In [1]:
import h2o
import pandas as pd
from h2o.automl import H2OAutoML
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, f1_score

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.6" 2020-01-14; OpenJDK Runtime Environment (build 11.0.6+10-post-Ubuntu-1ubuntu119.10.1); OpenJDK 64-Bit Server VM (build 11.0.6+10-post-Ubuntu-1ubuntu119.10.1, mixed mode, sharing)
  Starting server from /home/eugene/anaconda3/envs/twitter-bot-detection/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmplnc95t35
  JVM stdout: /tmp/tmplnc95t35/h2o_eugene_started_from_python.out
  JVM stderr: /tmp/tmplnc95t35/h2o_eugene_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O cluster uptime:,01 secs
H2O cluster timezone:,Europe/Kiev
H2O data parsing timezone:,UTC
H2O cluster version:,3.28.0.3
H2O cluster version age:,1 month and 7 days
H2O cluster name:,H2O_from_python_eugene_sp7jii
H2O cluster total nodes:,1
H2O cluster free memory:,3.893 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"accepting new members, healthy"


In [3]:
user_features = pd.read_pickle("data/04_features/user_features.pkl")
tweets_features = pd.read_pickle("data/04_features/tweets_features.pkl")

df = pd.concat([user_features, tweets_features], axis=1, join='inner')
df.fillna(0, inplace=True)
train, test = train_test_split(df, test_size=0.2, random_state=1, stratify=df['label'])

In [4]:
hf = h2o.H2OFrame(train)
hf["label"] = hf["label"].asfactor()

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [5]:
y = 'label'
x = hf.columns

In [23]:
aml = H2OAutoML(seed = 1)
aml.train(x = x, y = y, training_frame = hf)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [24]:
lb = aml.leaderboard
lb.head()

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
XGBoost_grid__1_AutoML_20200313_132941_model_9,0.972842,0.180771,0.881794,0.0854645,0.229552,0.0526942
XGBoost_grid__1_AutoML_20200313_132941_model_8,0.972505,0.181242,0.902011,0.085519,0.229822,0.0528181
XGBoost_2_AutoML_20200313_132941,0.972094,0.182855,0.894582,0.089526,0.23107,0.0533932
XGBoost_1_AutoML_20200313_132941,0.972028,0.182225,0.897274,0.0868696,0.230488,0.0531245
XGBoost_grid__1_AutoML_20200313_132941_model_3,0.972018,0.183118,0.904511,0.0875735,0.231341,0.0535185
StackedEnsemble_AllModels_AutoML_20200313_132941,0.971971,0.193765,0.898482,0.0887338,0.233401,0.0544759
XGBoost_grid__1_AutoML_20200313_132941_model_4,0.971368,0.184117,0.880023,0.0867986,0.231023,0.0533717
XGBoost_3_AutoML_20200313_132941,0.97118,0.185653,0.88278,0.0879385,0.232842,0.0542156
XGBoost_grid__1_AutoML_20200313_132941_model_1,0.970938,0.186173,0.926074,0.0886588,0.233194,0.0543794
XGBoost_grid__1_AutoML_20200313_132941_model_7,0.970857,0.18573,0.886882,0.0911532,0.232562,0.0540849


In [ ]:
h2o.save_model(aml.leader, 'data/06_models/')

In [25]:
pred = aml.leader.predict(h2o.H2OFrame(test.drop(columns=["label"])))

Parse progress: |█████████████████████████████████████████████████████████| 100%
xgboost prediction progress: |████████████████████████████████████████████| 100%


In [26]:
print(classification_report(test["label"], pred["predict"].as_data_frame().values, digits=5))

              precision    recall  f1-score   support

           0    0.94222   0.95778   0.94993      4903
           1    0.91104   0.88040   0.89546      2408

    accuracy                        0.93229      7311
   macro avg    0.92663   0.91909   0.92270      7311
weighted avg    0.93195   0.93229   0.93199      7311

